# Get Voices

In [ ]:
from pyneuphonic import Neuphonic
import os

client = Neuphonic(api_key=os.environ.get('NEUPHONIC_API_TOKEN'))
voices = client.voices.get()  # get's all available voices

for voice in voices:
    print(voice)

# Text To Speech

In [6]:
from pyneuphonic import Neuphonic, TTSConfig
from pyneuphonic.player import AudioPlayer
import os

client = Neuphonic(api_key=os.environ.get('NEUPHONIC_API_TOKEN'))
sse = client.tts.SSEClient()
tts_config = TTSConfig(speed=1.05,  voice='8e9c4bc8-3979-48ab-8626-df53befc2090', model="neu_hq")

with AudioPlayer() as player:
    response = sse.send("""Neuphonic generates high quality and low latency text to speech. Experience the speed and clarity of our dynamically powered voice synthesis by entering your text!""", tts_config=tts_config)
    player.play(response)
    # player.save_audio('output.wav') 

# Altering Speech Parameters

In [4]:
from pyneuphonic import Neuphonic, TTSConfig
from pyneuphonic.player import AudioPlayer
import os

client = Neuphonic(api_key=os.environ.get('NEUPHONIC_API_TOKEN'))
sse = client.tts.SSEClient()
tts_config = TTSConfig(speed=1.3,  
                       voice='8e9c4bc8-3979-48ab-8626-df53befc2090', 
                       model="neu_hq")

with AudioPlayer() as player:
    response = sse.send("""Neuphonic generates high quality and low latency text to speech. Experience the speed and clarity of our dynamically powered voice synthesis by entering your text!""", tts_config=tts_config)
    player.play(response)
    # player.save_audio('output.wav') 

# Voice Cloning

In [ ]:
from pyneuphonic import Neuphonic
import os

client = Neuphonic(api_key=os.environ.get('NEUPHONIC_API_TOKEN'))

voice_file_path = 'sample.wav'

result = client.voices.clone(voice_name = 'Snoop7',  
                             voice_tags = ['American', '50s'], 
                             voice_file_path = voice_file_path)

print(result['data'])

In [10]:
tts_config = TTSConfig(speed=1.05, model="neu_hq", voice_id=result['data']['id'])

sse = client.tts.SSEClient()

with AudioPlayer() as player:
    response = sse.send(
        "Neuphonic generates high quality and low latency text to speech. Experience the speed and clarity of our dynamically powered voice synthesis by entering your text!", 
        tts_config=tts_config
    )
    player.play(response)
    # player.save_audio('output.wav') 

# Speech to Speech

Run the example below to create an agent that will greet you and initiate a voice conversation. 
Speak back to it to initiate a response.

In [ ]:
import os
import asyncio

from pyneuphonic import Neuphonic, Agent


client = Neuphonic(api_key=os.environ.get('NEUPHONIC_API_TOKEN'))

agent_id = client.agents.create(
    name='Agent 1',
    prompt='You are a helpful agent. Answer in 10 words or less.',
    greeting='Hi, how can I help you today?'
)['data']['id']

agent = Agent(client, agent_id=agent_id, tts_model='neu_hq')

await agent.start()